In [1]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard

In [2]:
from datetime import datetime
from packaging import version

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
    
print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.0 or above."

import tensorboard
tensorboard.__version__

2023-09-04 09:35:15.527122: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


TensorFlow version:  2.12.0


'2.12.1'

In [3]:
# Simulated weather data (replace with your own dataset)
num_samples = 1000
sequence_length = 47  # Time steps per sequence
num_features = 5     # Weather features (e.g., temperature, humidity, pressure, etc.)


In [4]:
# Generate synthetic weather data (replace with actual data loading)
weather_data = np.random.rand(num_samples, sequence_length, num_features)
print(f"weather_data shape: {weather_data.shape}")

weather_data shape: (1000, 47, 5)


In [5]:

# Define a hybrid model with multi-scale attention
class MultiScaleAttentionModel(tf.keras.Model):
    def __init__(self, num_heads, num_layers, num_filters):
        super(MultiScaleAttentionModel, self).__init__()
        self.conv1d = layers.Conv1D(num_filters, kernel_size=3, padding='same', activation='relu')
        self.multi_head_attention = layers.MultiHeadAttention(num_heads=num_heads, key_dim=num_features, value_dim=num_features)  # Adjust key_dim and value_dim
        self.feed_forward = layers.Dense(num_filters, activation='relu')
        self.layer_norm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layer_norm2 = layers.LayerNormalization(epsilon=1e-6)
        
    def call(self, x):
        # print(x.shape)
        # Convolutional layer to capture local information
        x_conv = self.conv1d(x)
        print("1------")
        # Multi-head attention to capture global information
        x_attention = self.multi_head_attention(x_conv, x_conv, x_conv)
        print(f"{x_attention.shape}")
        x_attention = self.conv1d(x_attention)  # Additional convolution to match dimensions
        print(f"{x_attention.shape}")
        print(f"2--------------:{x.shape}, {x_attention.shape}")
        # Residual connection and layer normalization
        x_residual = self.layer_norm1(x + x_attention)
        print("3-------------------------")
        # Feed-forward network
        x_ffn = self.feed_forward(x_residual)
        print("4------------------------------")
        # Residual connection and layer normalization
        x_out = self.layer_norm2(x_residual + x_ffn)
        print("5------------------------------")
        print(x_out.shape)
        return x_out


In [6]:
# Build the multi-scale attention model
num_heads = 8
num_layers = 4
num_filters = 64

model = MultiScaleAttentionModel(num_heads, num_layers, num_filters)


2023-09-04 09:35:16.926880: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [7]:
# Compile the model (customize as needed)
model.compile(optimizer='adam', loss='mse')

# Train the model (replace with your own training data and settings)
x_train = weather_data[:, :-1, :]
y_train = weather_data[:, 1:, :]  # Predict the next time step

print(f"x train shape: {x_train.shape}")
print(f"y train shape: {y_train.shape}")


# Define the Keras TensorBoard callback.
logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)



model.fit(x_train, y_train, epochs=10, batch_size=32)
 #, callbacks=[tensorboard_callback])


# Use the trained model for weather forecasting
forecast = model.predict(x_train[:1])  # Make a forecast for the first sequence

print("Forecasted Weather:")
print(forecast)

x train shape: (1000, 46, 5)
y train shape: (1000, 46, 5)
Epoch 1/10
1------
(None, 46, 64)


ValueError: in user code:

    File "/home/xianjia/Workspace/miniconda3/envs/uwb-lstm-env/lib/python3.11/site-packages/keras/engine/training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "/home/xianjia/Workspace/miniconda3/envs/uwb-lstm-env/lib/python3.11/site-packages/keras/engine/training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/xianjia/Workspace/miniconda3/envs/uwb-lstm-env/lib/python3.11/site-packages/keras/engine/training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "/home/xianjia/Workspace/miniconda3/envs/uwb-lstm-env/lib/python3.11/site-packages/keras/engine/training.py", line 1050, in train_step
        y_pred = self(x, training=True)
    File "/home/xianjia/Workspace/miniconda3/envs/uwb-lstm-env/lib/python3.11/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(f